# 01 Logistic Recession Classifier

Logistic regression, ROC/PR, and threshold tuning.


## Table of Contents
- [Train/test split](#train-test-split)
- [Fit logistic](#fit-logistic)
- [ROC/PR](#roc-pr)
- [Threshold tuning](#threshold-tuning)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Classification notebooks turn the recession label into a **probability model**.
You will learn how to evaluate rare-event prediction and how to choose thresholds intentionally.


## Prerequisites (Quick Self-Check)
- Completed Part 02 (regression basics) or equivalent.
- Comfort interpreting probabilities and trade-offs (false positives vs false negatives).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Reporting only accuracy on imbalanced data.
- Using threshold=0.5 by default without considering costs.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/03_classification/01_logistic_recession_classifier.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/03_classification/01_logistic_recession_classifier.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Train a logistic regression classifier for next-quarter technical recession.

You will learn:
- how to do a time-based split
- how to fit a probabilistic classifier (outputs probabilities)
- how to interpret coefficients as log-odds / odds ratios
- why threshold selection is a decision problem, not a default



## Primer: sklearn pipelines (how to avoid preprocessing leakage)

Pipelines prevent a common mistake: fitting preprocessing (scaling, imputation) using information from the test set.

### Why pipelines exist (in one sentence)

> A `Pipeline` ensures that transformations are fit on training data only, then applied to test data.

### The key APIs

- `fit(X, y)`: learn parameters (scaler mean/std, model weights) from training.
- `transform(X)`: apply learned transform to new data.
- `fit_transform(X, y)`: convenience for training data only.

### Minimal pattern (classification)

```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
  ("scaler", StandardScaler()),
  ("model", LogisticRegression(max_iter=5000)),
])

# clf.fit(X_train, y_train)
# y_prob = clf.predict_proba(X_test)[:, 1]
```

**Expected output / sanity check**
- you never call `scaler.fit` on the full dataset
- you split by time first, then fit the pipeline on train

### Mini demo: the leakage you’re avoiding (toy)

```python
import numpy as np
from sklearn.preprocessing import StandardScaler

rng = np.random.default_rng(0)
X_train = rng.normal(loc=0.0, scale=1.0, size=(100, 1))
X_test  = rng.normal(loc=2.0, scale=1.0, size=(25, 1))

# WRONG: fit on train+test (leaks the future)
sc_wrong = StandardScaler().fit(np.vstack([X_train, X_test]))
X_test_wrong = sc_wrong.transform(X_test)

# RIGHT: fit on train only
sc_right = StandardScaler().fit(X_train)
X_test_right = sc_right.transform(X_test)

print("test mean after wrong scaling:", float(X_test_wrong.mean()))
print("test mean after right scaling:", float(X_test_right.mean()))
```

### Common pitfalls

- Splitting after preprocessing (leakage).
- Using random splits on time-indexed data (temporal leakage).
- Forgetting `ColumnTransformer` for mixed numeric/categorical columns (if needed).


<a id="train-test-split"></a>
## Train/test split

### Goal
Split chronologically so the model trains on the past and is evaluated on the future.



### Your Turn (1): Load data and select columns


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

y_col = 'target_recession_next_q'
x_cols = ['T10Y2Y_lag1', 'UNRATE_lag1', 'FEDFUNDS_lag1', 'INDPRO_lag1']

df_m = df[[y_col] + x_cols].dropna().copy()
df_m.tail()



### Your Turn (2): Create a time split


In [ ]:
from src.evaluation import time_train_test_split_index

split = time_train_test_split_index(len(df_m), test_size=0.2)
train = df_m.iloc[split.train_slice]
test = df_m.iloc[split.test_slice]

X_train = train[x_cols]
y_train = train[y_col].astype(int)
X_test = test[x_cols]
y_test = test[y_col].astype(int)

train.index.max(), test.index.min()



<a id="fit-logistic"></a>
## Fit logistic

### Goal
Fit a logistic regression model inside a Pipeline (to avoid preprocessing leakage).



### Your Turn (1): Fit the pipeline


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=5000)),
])

# TODO: Fit on training data
clf.fit(X_train, y_train)

# Predicted probabilities for class 1
p_test = clf.predict_proba(X_test)[:, 1]
p_test[:5]



### Your Turn (2): Evaluate metrics


In [ ]:
from src.evaluation import classification_metrics

classification_metrics(y_test.to_numpy(), p_test, threshold=0.5)



### Your Turn (3): Interpret coefficients (odds ratios)


In [ ]:
import numpy as np
import pandas as pd

# Coefficients live in the underlying model
coefs = clf.named_steps['model'].coef_[0]

# TODO: Build a coefficient table.
coef_df = pd.DataFrame({'feature': x_cols, 'coef': coefs})

# Odds ratio for a 1-unit increase in standardized feature:
# OR = exp(coef)
coef_df['odds_ratio'] = np.exp(coef_df['coef'])
coef_df.sort_values('coef')



### Logistic vs Probit and marginal effects

**Logistic vs Probit**: Both model binary outcomes. Logistic regression uses the logistic (sigmoid) link function; probit uses the normal CDF. In practice they give very similar predictions. Logistic is more common in ML (odds ratio interpretation); probit is traditional in econometrics. Both are available in `statsmodels`:

```python
import statsmodels.api as sm
# Logistic
sm.Logit(y, X).fit()
# Probit
sm.Probit(y, X).fit()
```

**Marginal effects**: The logistic coefficient $\beta_j$ is a change in *log-odds*, not a change in probability. The actual effect on $P(y=1)$ depends on where you are on the sigmoid curve. To get interpretable probability changes, compute **average marginal effects (AME)**:

```python
res_logit = sm.Logit(y_train, sm.add_constant(X_train)).fit()
mfx = res_logit.get_margeff(at='overall')
print(mfx.summary())
```

AME tells you: "On average across the sample, a one-unit increase in $x_j$ is associated with a ___ percentage point change in $P(y=1)$." This is more interpretable than the raw coefficient for decision-making.

### Your Turn (optional): Compute marginal effects

In [ ]:
# TODO (optional): Fit a statsmodels Logit and compute average marginal effects.
# Compare the AME to the sklearn odds ratios above.
# import statsmodels.api as sm
# X_tr_c = sm.add_constant(X_train.astype(float))
# res_logit = sm.Logit(y_train.astype(float), X_tr_c).fit(disp=0)
# mfx = res_logit.get_margeff(at='overall')
# print(mfx.summary())
...

<a id="roc-pr"></a>
## ROC/PR

### Goal
Plot ROC and precision-recall curves.

Why both?
- ROC can look optimistic under heavy class imbalance
- PR focuses on the positive class (recessions)



### Your Turn: Plot ROC and PR curves


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

# TODO: Create ROC and PR plots.
...



<a id="threshold-tuning"></a>
## Threshold tuning

### Goal
Compare metrics at different probability thresholds.

A lower threshold catches more recessions (higher recall) but raises false positives.



### Your Turn: Evaluate multiple thresholds


In [ ]:
from src.evaluation import classification_metrics

for thr in [0.3, 0.5, 0.7]:
    m = classification_metrics(y_test.to_numpy(), p_test, threshold=thr)
    print(thr, m)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Train/test split</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_logistic_recession_classifier — Train/test split
import pandas as pd
import numpy as np

df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True).dropna()
target = 'target_recession_next_q'
drop_cols = {target, 'recession', 'GDPC1', 'gdp_growth_qoq', 'gdp_growth_qoq_annualized', 'gdp_growth_yoy'}
X = df[[c for c in df.columns if c not in drop_cols]].astype(float)
y = df[target].astype(int)
split = int(len(df)*0.8)
X_tr, X_te = X.iloc[:split], X.iloc[split:]
y_tr, y_te = y.iloc[:split], y.iloc[split:]
```

</details>

<details><summary>Solution: Fit logistic</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_logistic_recession_classifier — Fit logistic
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from src import evaluation

clf = Pipeline([('scaler', StandardScaler()), ('m', LogisticRegression(max_iter=5000))])
clf.fit(X_tr, y_tr)
p = clf.predict_proba(X_te)[:,1]
evaluation.classification_metrics(y_te.to_numpy(), p)
```

</details>

<details><summary>Solution: ROC/PR</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_logistic_recession_classifier — ROC/PR
from sklearn.metrics import roc_curve, precision_recall_curve
from src import plots

fpr, tpr, _ = roc_curve(y_te, p)
prec, rec, _ = precision_recall_curve(y_te, p)
plots.plot_roc_curve(fpr, tpr)
plots.plot_pr_curve(rec, prec)
```

</details>

<details><summary>Solution: Threshold tuning</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_logistic_recession_classifier — Threshold tuning
from src import evaluation
for thr in [0.3, 0.5, 0.7]:
    print(thr, evaluation.classification_metrics(y_te.to_numpy(), p, threshold=thr))
```

</details>

